In [14]:
import transformers
import torch
import os
import json
import random
import numpy as np
import argparse
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from tqdm import tqdm
from torch.nn import DataParallel
from tokenizations.bpe_tokenizer import get_encoder



def build_files(data_path, tokenized_data_path, full_tokenizer, min_length):
    with open(data_path, 'r', encoding='utf8') as f:
        print('reading lines')
        lines = json.load(f)
        lines = [line.replace('\n', ' [SEP] ') for line in lines]  # 用[SEP]表示换行, 段落之间使用SEP表示段落结束

    if not os.path.exists(tokenized_data_path):
        os.mkdir(tokenized_data_path)
    
    num_pieces = len(lines)
    for i in tqdm(range(num_pieces)):
        sublines = lines[all_len // num_pieces * i: all_len // num_pieces * (i + 1)]

        if i == num_pieces - 1:
            sublines.extend(lines[all_len // num_pieces * (i + 1):])  # 把尾部例子添加到最后一个piece
        sublines = [full_tokenizer.tokenize(line) for line in sublines if len(line) > min_length]  # 只考虑长度超过 min_length 的句子
        print(sublines)
        sublines = [full_tokenizer.convert_tokens_to_ids(line) for line in sublines]
        full_line = []
        for subline in sublines:
            full_line.append(full_tokenizer.convert_tokens_to_ids('[MASK]'))  # 文章开头添加MASK表示文章开始
            full_line.extend(subline)
            full_line.append(full_tokenizer.convert_tokens_to_ids('[CLS]'))  # 文章之间添加CLS表示文章结束
        with open(tokenized_data_path + 'tokenized_train_{}.txt'.format(i), 'w') as f:
            for id in full_line:
                f.write(str(id) + ' ')
    print('finish')


In [15]:
data_path = 'data/train_test.json'
tokenized_data_path = 'data/tokenized_test/'
num_pieces = 5
from tokenizations import tokenization_bert
full_tokenizer = tokenization_bert.BertTokenizer(vocab_file='./cache/vocab_small.txt')
min_length = 4

In [16]:
build_files(data_path, tokenized_data_path, num_pieces, full_tokenizer, min_length)

reading lines
['（山西）国际业务受理主要包括国际漫游和台港澳漫游业务的开通流程及规则。具体业务规则如下：一、业务介绍：涉及国际漫游和台港澳漫游业务的开通。二、执行时间：2019年6月28日开始执行。三、适用范围：适用于山西联通移网用户，要求用户已实名制、非黑名单、无逾期欠费、无在途工单、手机开机并为正常状态。四、流程说明：（一）办理渠道：自有营业厅、客服热线10010、漫游客服热线+8618618610010、联通的短信营业厅、网上营业厅、中国联通APP、微信营业厅。（二）办理方式：自2020年5月12日起，全省执行所有用户办理国际、台港澳漫游业务免预存直接可办理，新入网用户默认开通国际、台港澳漫游业务。【表格内容】预付费用户可单独开通国际、台港澳长途业务，国际、台港澳漫游业务默认开通；后付费用户可单独开通国际、台港澳长途业务，但国际、台港澳漫游业务不可单独开通，需同时开通国际长途。（三）开通说明：1、已开通VPN的用户，如需办理开通国漫，无需关闭VPN。2、开通三方通话功能的用户，需要取消后办理，与国漫业务互斥。3、开通联通VoLTE业务会影响国际漫游业务的正常使用，如需开通，需要先取消国际漫游业务，待开通VoLTE业务后再办理该业务。（四）关闭说明：1、用户需持相关有效证件到联通自有营业厅申请关闭国际、台港澳漫游业务权限。2、个人用户需携带有效身份证件或经办人携带经办人及机主本人有效身份证件；单位用户需携带经办人有效证件及加盖公章的公函。', '“快乐驿站”（1169781611）是由北京畅捷科技有限公司提供的业务，主要为用户提供淘米网米币充值和卡密获取等小额支付类业务。版权归属北京时代博纳科技有限公司。业务规则方面：-计费规则：按次计费，信息费为8元/次，不包括通信费。-生效规则：原文中未提及具体生效规则。-退订规则：按次计费的业务无需退订。用户在拨打1169781611后，会在6秒音乐后听到滴声，此时开始计费。在滴声之前不会收取信息费，且如果不进行拨打，则不会产生任何费用。', '保号通是一种提供手机数据、网络存储间数据交互及隐私通讯功能的业务，主要面向山西用户，具有通讯录备份与恢复、来电提醒/短信转移、手机网盘、即时分享等多样化功能，并分为体验版、时尚版和全能版，以适应不同用户的需求。（山西）保号通（10655816）提供的资费档位有：1.

100%|██████████| 5/5 [00:00<00:00, 854.41it/s]

['（山西）国际业务受理主要包括国际漫游和台港澳漫游业务的开通流程及规则。具体业务规则如下：一、业务介绍：涉及国际漫游和台港澳漫游业务的开通。二、执行时间：2019年6月28日开始执行。三、适用范围：适用于山西联通移网用户，要求用户已实名制、非黑名单、无逾期欠费、无在途工单、手机开机并为正常状态。四、流程说明：（一）办理渠道：自有营业厅、客服热线10010、漫游客服热线+8618618610010、联通的短信营业厅、网上营业厅、中国联通APP、微信营业厅。（二）办理方式：自2020年5月12日起，全省执行所有用户办理国际、台港澳漫游业务免预存直接可办理，新入网用户默认开通国际、台港澳漫游业务。【表格内容】预付费用户可单独开通国际、台港澳长途业务，国际、台港澳漫游业务默认开通；后付费用户可单独开通国际、台港澳长途业务，但国际、台港澳漫游业务不可单独开通，需同时开通国际长途。（三）开通说明：1、已开通VPN的用户，如需办理开通国漫，无需关闭VPN。2、开通三方通话功能的用户，需要取消后办理，与国漫业务互斥。3、开通联通VoLTE业务会影响国际漫游业务的正常使用，如需开通，需要先取消国际漫游业务，待开通VoLTE业务后再办理该业务。（四）关闭说明：1、用户需持相关有效证件到联通自有营业厅申请关闭国际、台港澳漫游业务权限。2、个人用户需携带有效身份证件或经办人携带经办人及机主本人有效身份证件；单位用户需携带经办人有效证件及加盖公章的公函。']
1
[['（', '山', '西', '）', '国', '际', '业', '务', '受', '理', '主', '要', '包', '括', '国', '际', '漫', '游', '和', '台', '港', '澳', '漫', '游', '业', '务', '的', '开', '通', '流', '程', '及', '规', '则', '。', '具', '体', '业', '务', '规', '则', '如', '下', '：', '一', '、', '业', '务', '介', '绍', '：', '涉', '及', '国', '际', '漫', '游', '和', '台', '港', '澳', '漫', '游', '业', '务', '的', '开', '通', '。', '二', '、', '执', '行', '时'

In [60]:
with open('./data/tokenized/tokenized_train_21.txt', 'r') as f:
    lines = f.readlines()
    # print(lines)


In [61]:
lines[0].split()

['103',
 '1962',
 '832',
 '6369',
 '2571',
 '5470',
 '3221',
 '671',
 '702',
 '3255',
 '2716',
 '4500',
 '2339',
 '3302',
 '1218',
 '2398',
 '1378',
 '8024',
 '809',
 '1920',
 '3144',
 '2945',
 '116',
 '10362',
 '5143',
 '5320',
 '3255',
 '5543',
 '1276',
 '6981',
 '821',
 '689',
 '4500',
 '2339',
 '818',
 '1218',
 '1469',
 '952',
 '6848',
 '782',
 '8024',
 '3192',
 '1762',
 '2141',
 '4385',
 '821',
 '689',
 '856',
 '2768',
 '3315',
 '4130',
 '3833',
 '4500',
 '2339',
 '1469',
 '5632',
 '4507',
 '5466',
 '689',
 '5442',
 '2571',
 '6862',
 '2218',
 '689',
 '511',
 '2124',
 '2990',
 '897',
 '1259',
 '2886',
 '1825',
 '4794',
 '3302',
 '1218',
 '1963',
 '2875',
 '2339',
 '510',
 '818',
 '1218',
 '1355',
 '2357',
 '510',
 '1276',
 '6981',
 '4488',
 '1166',
 '510',
 '818',
 '1218',
 '7741',
 '3119',
 '510',
 '702',
 '4925',
 '807',
 '2519',
 '8024',
 '809',
 '1350',
 '2137',
 '1169',
 '1265',
 '782',
 '1213',
 '6598',
 '3975',
 '6237',
 '1104',
 '3175',
 '3428',
 '5023',
 '8024',
 '1221',
 

In [62]:
token_list = lines[0].split()
for i in range(len(token_list)):
    try:
        print(full_tokenizer.convert_ids_to_tokens(int(token_list[i])))
    except:
        print()
# full_tokenizer.convert_ids_to_tokens(int(lines[0][3]))

[MASK]
好
伙
计
快
聘
是
一
个
智
慧
用
工
服
务
平
台
，
以
大
数
据
+
saas
系
统
智
能
匹
配
企
业
用
工
任
务
和
候
选
人
，
旨
在
实
现
企
业
低
成
本
灵
活
用
工
和
自
由
职
业
者
快
速
就
业
。
它
提
供
包
括
基
础
服
务
如
招
工
、
任
务
发
布
、
匹
配
甄
别
、
任
务
验
收
、
个
税
代
征
，
以
及
定
制
化
人
力
资
源
解
决
方
案
等
，
助
力
企
业
降
本
增
效
和
个
人
合
规
完
税
。
好
伙
计
快
聘
的
资
费
档
位
主
要
包
括
以
下
几
种
：
1
.
基
础
套
餐
：
4
8
0
0
元
/
1
2
月
，
提
供
企
业
在
好
伙
计
快
聘
智
慧
用
工
服
务
平
台
招
工
、
任
务
发
布
、
匹
配
甄
别
、
任
务
验
收
、
个
税
代
征
等
基
础
服
务
。
2
.
专
业
版
套
餐
：
1
2
0
0
0
元
/
1
2
月
，
提
供
基
础
服
务
外
，
还
包
括
一
年
纯
人
工
服
务
。
3
.
政
企
人
力
资
源
服
务
包
：
企
业
定
制
化
人
力
资
源
解
决
方
案
，
价
格
根
据
用
户
具
体
需
求
定
价
，
如
为
小
型
企
业
提
供
财
税
管
理
针
对
性
的
解
决
方
案
，
预
估
2
人
，
3
天
完
成
，
价
格
为
6
0
0
0
元
。
4
.
政
企
人
力
资
源
服
务
包
：
企
业
税
务
筹
划
培
训
，
价
格
根
据
培
训
规
模
、
培
训
内
容
及
培
训
形
式
定
价
。
5
.
政
企
人
力
资
源
服
务
包
：
公
共
部
门
人
力
资
源
开
发
与
管
理
培
训
，
价
格
根
据
培
训
规
模
、
培
训
内
容
及
培
训
形
式
定
价
。
6
.
政
企
人
力
资
源
服
务
包
：
政
府
部
门
人
力
资
源
管
理
实
践
活
动
解
决
方
案
，
价
格
根
据
用
户
具
体
需
求
定
价
。
7
.
好
伙
计
快
聘
咨
询
服
务
：
5
0
0
元
/
次
，
针
对
财
税
管
理
、
财
务
指
导
、
财
税


In [55]:
token_list

['103',
 '8020',
 '2255',
 '6205',
 '8021',
 '1744',
 '7354',
 '689',
 '1218',
 '1358',
 '4415',
 '712',
 '6206',
 '1259',
 '2886',
 '1744',
 '7354',
 '4035',
 '3952',
 '1469',
 '1378',
 '3949',
 '4078',
 '4035',
 '3952',
 '689',
 '1218',
 '4638',
 '2458',
 '6858',
 '3837',
 '4923',
 '1350',
 '6226',
 '1156',
 '511',
 '1072',
 '860',
 '689',
 '1218',
 '6226',
 '1156',
 '1963',
 '678',
 '8038',
 '671',
 '510',
 '689',
 '1218',
 '792',
 '5305',
 '8038',
 '3868',
 '1350',
 '1744',
 '7354',
 '4035',
 '3952',
 '1469',
 '1378',
 '3949',
 '4078',
 '4035',
 '3952',
 '689',
 '1218',
 '4638',
 '2458',
 '6858',
 '511',
 '753',
 '510',
 '2809',
 '6121',
 '3198',
 '7313',
 '8038',
 '123',
 '121',
 '122',
 '130',
 '2399',
 '127',
 '3299',
 '123',
 '129',
 '3189',
 '2458',
 '1993',
 '2809',
 '6121',
 '511',
 '676',
 '510',
 '6844',
 '4500',
 '5745',
 '1741',
 '8038',
 '6844',
 '4500',
 '754',
 '2255',
 '6205',
 '5468',
 '6858',
 '4919',
 '5381',
 '4500',
 '2787',
 '8024',
 '6206',
 '3724',
 '4500',
 

In [40]:
lines[0][20]

'0'

In [47]:
lines[0][7]

'0'

In [66]:
data_path = 'data/train_test.json'
tokenized_data_path = 'data/tokenized_test1/'
num_pieces = 6

# def build_files(data_path, tokenized_data_path, num_pieces, full_tokenizer, min_length):
with open(data_path, 'r', encoding='utf8') as f:
    print('reading lines')
    lines = json.load(f)
    lines = [line.replace('\n', ' [SEP] ') for line in lines]  # 用[SEP]表示换行, 段落之间使用SEP表示段落结束
print(lines)
all_len = len(lines)
print('all_len',all_len)
if not os.path.exists(tokenized_data_path):
    os.mkdir(tokenized_data_path)
for i in tqdm(range(num_pieces)):
    print('i',i)
    sublines = lines[all_len // num_pieces * i: all_len // num_pieces * (i + 1)]
    print(sublines)
    if i == num_pieces - 1:
        sublines.extend(lines[all_len // num_pieces * (i + 1):])  # 把尾部例子添加到最后一个piece

    print('len(sublines)',len(sublines))

    sublines = [full_tokenizer.tokenize(line) for line in sublines if
                len(line) > min_length]  # 只考虑长度超过min_length的句子
    print(sublines)
    sublines = [full_tokenizer.convert_tokens_to_ids(line) for line in sublines]
    full_line = []
        # for subline in sublines:
    #         full_line.append(full_tokenizer.convert_tokens_to_ids('[MASK]'))  # 文章开头添加MASK表示文章开始
    #         full_line.extend(subline)
    #         full_line.append(full_tokenizer.convert_tokens_to_ids('[CLS]'))  # 文章之间添加CLS表示文章结束
    #     with open(tokenized_data_path + 'tokenized_train_{}.txt'.format(i), 'w') as f:
    #         for id in full_line:
    #             f.write(str(id) + ' ')
    # print('finish')


reading lines
['（山西）国际业务受理主要包括国际漫游和台港澳漫游业务的开通流程及规则。具体业务规则如下：一、业务介绍：涉及国际漫游和台港澳漫游业务的开通。二、执行时间：2019年6月28日开始执行。三、适用范围：适用于山西联通移网用户，要求用户已实名制、非黑名单、无逾期欠费、无在途工单、手机开机并为正常状态。四、流程说明：（一）办理渠道：自有营业厅、客服热线10010、漫游客服热线+8618618610010、联通的短信营业厅、网上营业厅、中国联通APP、微信营业厅。（二）办理方式：自2020年5月12日起，全省执行所有用户办理国际、台港澳漫游业务免预存直接可办理，新入网用户默认开通国际、台港澳漫游业务。【表格内容】预付费用户可单独开通国际、台港澳长途业务，国际、台港澳漫游业务默认开通；后付费用户可单独开通国际、台港澳长途业务，但国际、台港澳漫游业务不可单独开通，需同时开通国际长途。（三）开通说明：1、已开通VPN的用户，如需办理开通国漫，无需关闭VPN。2、开通三方通话功能的用户，需要取消后办理，与国漫业务互斥。3、开通联通VoLTE业务会影响国际漫游业务的正常使用，如需开通，需要先取消国际漫游业务，待开通VoLTE业务后再办理该业务。（四）关闭说明：1、用户需持相关有效证件到联通自有营业厅申请关闭国际、台港澳漫游业务权限。2、个人用户需携带有效身份证件或经办人携带经办人及机主本人有效身份证件；单位用户需携带经办人有效证件及加盖公章的公函。', '“快乐驿站”（1169781611）是由北京畅捷科技有限公司提供的业务，主要为用户提供淘米网米币充值和卡密获取等小额支付类业务。版权归属北京时代博纳科技有限公司。业务规则方面：-计费规则：按次计费，信息费为8元/次，不包括通信费。-生效规则：原文中未提及具体生效规则。-退订规则：按次计费的业务无需退订。用户在拨打1169781611后，会在6秒音乐后听到滴声，此时开始计费。在滴声之前不会收取信息费，且如果不进行拨打，则不会产生任何费用。', '保号通是一种提供手机数据、网络存储间数据交互及隐私通讯功能的业务，主要面向山西用户，具有通讯录备份与恢复、来电提醒/短信转移、手机网盘、即时分享等多样化功能，并分为体验版、时尚版和全能版，以适应不同用户的需求。（山西）保号通（10655816）提供的资费档位有：1.

100%|██████████| 6/6 [00:00<00:00, 772.22it/s]

i 0
['（山西）国际业务受理主要包括国际漫游和台港澳漫游业务的开通流程及规则。具体业务规则如下：一、业务介绍：涉及国际漫游和台港澳漫游业务的开通。二、执行时间：2019年6月28日开始执行。三、适用范围：适用于山西联通移网用户，要求用户已实名制、非黑名单、无逾期欠费、无在途工单、手机开机并为正常状态。四、流程说明：（一）办理渠道：自有营业厅、客服热线10010、漫游客服热线+8618618610010、联通的短信营业厅、网上营业厅、中国联通APP、微信营业厅。（二）办理方式：自2020年5月12日起，全省执行所有用户办理国际、台港澳漫游业务免预存直接可办理，新入网用户默认开通国际、台港澳漫游业务。【表格内容】预付费用户可单独开通国际、台港澳长途业务，国际、台港澳漫游业务默认开通；后付费用户可单独开通国际、台港澳长途业务，但国际、台港澳漫游业务不可单独开通，需同时开通国际长途。（三）开通说明：1、已开通VPN的用户，如需办理开通国漫，无需关闭VPN。2、开通三方通话功能的用户，需要取消后办理，与国漫业务互斥。3、开通联通VoLTE业务会影响国际漫游业务的正常使用，如需开通，需要先取消国际漫游业务，待开通VoLTE业务后再办理该业务。（四）关闭说明：1、用户需持相关有效证件到联通自有营业厅申请关闭国际、台港澳漫游业务权限。2、个人用户需携带有效身份证件或经办人携带经办人及机主本人有效身份证件；单位用户需携带经办人有效证件及加盖公章的公函。']
len(sublines) 1
[['（', '山', '西', '）', '国', '际', '业', '务', '受', '理', '主', '要', '包', '括', '国', '际', '漫', '游', '和', '台', '港', '澳', '漫', '游', '业', '务', '的', '开', '通', '流', '程', '及', '规', '则', '。', '具', '体', '业', '务', '规', '则', '如', '下', '：', '一', '、', '业', '务', '介', '绍', '：', '涉', '及', '国', '际', '漫', '游', '和', '台', '港', '澳', '漫', '游', '业', '务', '的', '开', '通', '。', '二', 